In [10]:
# -----------------------------
# Install dependencies
# -----------------------------
!pip install datasets transformers --quiet

# -----------------------------
# Imports
# -----------------------------
import torch
import torch.nn as nn
from torch.nn import functional as F
from datasets import load_dataset
from transformers import GPT2TokenizerFast

# -----------------------------
# Hyperparameters
# -----------------------------
batch_size = 32
block_size = 128
max_iters = 5000        # train more for better results
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 100
n_embd = 256
n_head = 8
n_layer = 4
dropout = 0.1

torch.manual_seed(1337)

# -----------------------------
# Dataset (WikiText-103 subset)
# -----------------------------
print("Loading WikiText-103...")
ds = load_dataset("wikitext", "wikitext-103-raw-v1", split="train[:10%]")  # 10% for Colab
print(f"Loaded {len(ds)} documents")

# -----------------------------
# Tokenizer (GPT-2 BPE)
# -----------------------------
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

print("Tokenizing dataset...")
tokens = []
for item in ds:
    if item.get("text"):
        ids = tokenizer.encode(item["text"])
        tokens.extend(ids)
data = torch.tensor(tokens, dtype=torch.long)
print(f"Total tokens: {len(data):,}")

vocab_size = tokenizer.vocab_size
print(f"Vocab size: {vocab_size}")

# train/val split
n = int(0.9 * len(data))
train_data, val_data = data[:n], data[n:]

def get_batch(split):
    data_local = train_data if split == "train" else val_data
    ix = torch.randint(len(data_local) - block_size, (batch_size,))
    x = torch.stack([data_local[i:i+block_size] for i in ix])
    y = torch.stack([data_local[i+1:i+block_size+1] for i in ix])
    return x.to(device), y.to(device)

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ["train", "val"]:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            xb, yb = get_batch(split)
            _, loss = model(xb, yb)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# -----------------------------
# Model Components (with debug prints)
# -----------------------------
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2, -1) * (C ** -0.5)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float("-inf"))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.proj(out)
        out = self.dropout(out)
        return out

class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )
    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class GPT(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits.view(B*T, -1), targets.view(B*T))
        return logits, loss

    def generate(self, idx, max_new_tokens, temperature=1.0):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :] / temperature
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

# -----------------------------
# Init Model + Params
# -----------------------------
model = GPT().to(device)
param_count = sum(p.numel() for p in model.parameters())
print(f"Model params: {param_count/1e6:.2f}M ({param_count} parameters)")

# -----------------------------
# Training
# -----------------------------
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for it in range(max_iters):
    if it % eval_interval == 0 or it == max_iters - 1:
        losses = estimate_loss()
        print(f"step {it}: train {losses['train']:.4f}, val {losses['val']:.4f}")
        torch.save(model.state_dict(), "mini_gpt.pt")  # save checkpoints

    xb, yb = get_batch("train")
    _, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# -----------------------------
# Generate text
# -----------------------------
context = torch.zeros((1, 1), dtype=torch.long, device=device)
out_idx = model.generate(context, max_new_tokens=200, temperature=0.8)[0].tolist()
print("\n----- GENERATED TEXT -----\n")
print(tokenizer.decode(out_idx))


Loading WikiText-103...
Loaded 180135 documents
Tokenizing dataset...
Total tokens: 11,682,614
Vocab size: 50257
Model params: 28.97M (28971089 parameters)
step 0: train 10.9818, val 10.9814
step 500: train 6.4975, val 6.5305
step 1000: train 6.1014, val 6.1861
step 1500: train 5.8346, val 5.9657
step 2000: train 5.6652, val 5.8175
step 2500: train 5.4832, val 5.6948
step 3000: train 5.3682, val 5.5915
step 3500: train 5.2575, val 5.4960
step 4000: train 5.1576, val 5.4438
step 4500: train 5.0717, val 5.4083
step 4999: train 5.0130, val 5.3548

----- GENERATED TEXT -----

! @-@ class matches in the National League Baseball . Ulysseshiba was born in the Capitals in 1902 and 1960 , where he created a major role as editor of the University of the Women entitled Commandy . 
 In the late 1990s , the game was elected in the city of the 1930s in 1955 – 1950 , at Düh had been signed as the McDonald OR to also fell on 27 June 1939 . For the Nintendo 64 season , the team decided to the United St

In [11]:
prompt = "Artificial intelligence is changing the world"
input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

out_idx = model.generate(input_ids, max_new_tokens=200, temperature=0.8)[0].tolist()
print("\n----- GENERATED TEXT -----\n")
print(tokenizer.decode(out_idx))



----- GENERATED TEXT -----

Artificial intelligence is changing the world of the final standardopic herbivated 1772 in zinc . Despite the process is also known as a " less than the other " man " of the king and the imperial Isles song " . 
 = = = Critical reception = = = 
 = = = = = Others = 
 The music video nine South Korea 
 The eleventh version of Tokyo : 00 : the advert ( apart from the group of the first time ) . The comic strip was created a champion @-@ block at the muzzle velocity of 16 @.@ 5 inches ( 0 @,@ 000 m3 sq mi ) across the course of the 20th century . The original key , the Sun were contained for the second half @-@ ray . 
 = = = = = A total of 7 : 12 @-@ wide titles = = = = = 
 " At a UEFA 5 @-@ Flying " was the first segment of the few days in every six months . The castle was
